In [ ]:
import torch
import torch.nn as nn

from tqdm import tqdm
import wandb

from transformer import Transformer

In [ ]:
net = Transformer()
optim = torch.optim.AdamW(params = net.parameters())
loss_fn = nn.CrossEntropyLoss()
epochs=100
Xtest = torch.rand(20,200)
Ytest = torch.randint(0,4,(20,))
Xtrain = torch.rand(40,200)
Ytrain = torch.randint(0,4,(40,))

dataset = torch.utils.data.TensorDataset(Xtrain,Ytrain)
dataloader = torch.utils.data.DataLoader(dataset,batch_size=4,shuffle=True)

In [ ]:
wandb.init(
    project="cell_type_classification",
    config={}
)


for e in tqdm(range(epochs)):
    net.eval()
    predTest = net(Xtest)
    predTrain = net(Xtrain)
    lossTest = loss_fn(predTest,Ytest).item()
    loss_fn.zero_grad()
    lossTrain = loss_fn(predTrain,Ytrain).item()
    loss_fn.zero_grad()
    
    wandb.log({"train_loss": lossTrain, "test_loss": lossTest})

    net.train()
    for i, (batch_X, batch_Y) in enumerate(dataloader):
        pred = net(batch_X)
        loss = loss_fn(pred,batch_Y)
        loss.backward()
        optim.step()
        optim.zero_grad()